In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
from torch_geometric.data import Data, DataLoader
import matplotlib.pyplot as plt
import pandas as pd


ModuleNotFoundError: ignored

In [ ]:
### Neural net model, trained to predict a (6d representation of rod) from q (4x4x100 represetnation of rod)

In [ ]:
# say that data is in the format:
# q_n = 16x100
# a_n = 6x1 (or 1x6)
# input_data = list of 1000 tuples [(q_0, a_0), (q_1, a_1), (q_2, a_2), ... , (q_1000, a_1000)]

In [ ]:

train_data = input_data(0:800) # training data from csv/pandas (80% of existing data?)
test_data = input_data(800:-1) # test data from csv/pandas (remaining ~20% of data)

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=100)



Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16*100, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1600, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=6, bias=True)
    (5): ReLU()
  )
)


In [ ]:
### loss function and optimizer for training 
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Functions for training and testing the model
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
### Train the model
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300640  [    0/60000]
loss: 2.297163  [ 6400/60000]
loss: 2.289649  [12800/60000]
loss: 2.288036  [19200/60000]
loss: 2.260669  [25600/60000]
loss: 2.248673  [32000/60000]
loss: 2.245510  [38400/60000]
loss: 2.224563  [44800/60000]
loss: 2.222418  [51200/60000]
loss: 2.197868  [57600/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 0.034486 

Epoch 2
-------------------------------
loss: 2.211352  [    0/60000]
loss: 2.201629  [ 6400/60000]
loss: 2.169171  [12800/60000]
loss: 2.174231  [19200/60000]
loss: 2.120969  [25600/60000]
loss: 2.087647  [32000/60000]
loss: 2.103829  [38400/60000]
loss: 2.050085  [44800/60000]
loss: 2.056039  [51200/60000]
loss: 2.012417  [57600/60000]
Test Error: 
 Accuracy: 57.3%, Avg loss: 0.031577 

Epoch 3
-------------------------------
loss: 2.048275  [    0/60000]
loss: 2.025205  [ 6400/60000]
loss: 1.946805  [12800/60000]
loss: 1.971127  [19200/60000]
loss: 1.868205  [25600/60000]
loss: 1.815789  [32000/600

In [ ]:
### Save a model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
### Load a model
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model.pth"))

In [ ]:
### Use the model to make predictions
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
